## Docker pour Java

Cette partie illustre les différentes approches pour générer des images docker pour des applications Java.

### A partir d'une image de maven
Le plus simple est de s'appuyer sur une image de maven, d'y copier les sources et de donner comme commande le résutat de compilation.

```{literalinclude} sample-java/helloworld/Dockerfile.01.mavenimage
:language: dockerfile
:caption: sample-java/helloworld/Dockerfile.01.mavenimage
:name: sample-java/helloworld/Dockerfile.01.mavenimage
```

In [4]:
(cd sample-java/helloworld && \
docker image build --quiet --tag javahello:mavenimage --file Dockerfile.01.mavenimage . )

sha256:dd60669ea7ed38614f6694f98c6c572001f6587094376e06b34e99ea413f67cf


In [5]:
docker image ls|grep javahello|grep "mavenimage "|tr -s ' '

javahello mavenimage dd60669ea7ed 1 second ago 573MB


In [6]:
docker run --rm javahello:mavenimage

21:21:34.610 [fr.univtln.bruno.demos.docker.App.main()] INFO fr.univtln.bruno.demos.docker.App - Hello world maven + docker.


### Multistage

Il est plus efficace de s'appuyer sur la fabrication multistage d'une image docker pour séparer la chaîne de compilation de l'image d'exécution.
On utilise alors comme image finale une base de JRE (donc plus légère) dans laquelle uniquement le résultat de la compilation est copié (attention, il doit inclure les dépendances cf. uberjar). 

```{literalinclude} sample-java/helloworld/Dockerfile.02.mavenimagestage
:language: dockerfile
:caption: sample-java/helloworld/Dockerfile.02.mavenimagestage
:name: sample-java/helloworld/Dockerfile.02.mavenimagestage
```

In [10]:
(cd sample-java/helloworld && \
docker image build --quiet --tag javahello:mavenimagestage --file Dockerfile.02.mavenimagestage . )

sha256:0987f84ebd5796d364ef3ec6e19b1a9321eb7bd8f4e186c495d349eaee3dc3fb


In [11]:
docker image ls|grep javahello|grep mavenimagestage|tr -s ' '

javahello mavenimagestage 0987f84ebd57 Less than a second ago 262MB


In [12]:
docker run --rm javahello:mavenimagestage

21:22:39.682 [main] INFO fr.univtln.bruno.demos.docker.App - Hello world maven + docker.


### Cache docker

Les versions récentes de Docker proposent un mécanisme de cache qui permet de préciser explictement un dossier à mettre en cache lors du build d'une image. Pour Java et Maven, on peut ainsi mettre en cache `~/.m2` pour éviter de retélécharger systématiquement.

```{literalinclude} sample-java/helloworld/Dockerfile.03.cache
:language: dockerfile
:caption: sample-java/helloworld/Dockerfile.03.cache
:name: sample-java/helloworld/Dockerfile.03.cache
```

In [15]:
(cd sample-java/helloworld && \
docker image build --quiet --tag javahello:cache --file Dockerfile.03.cache . )

sha256:ce8587c004656fb519db772ea32508cdfab97753931b264ce6ece0138507f98a


In [16]:
docker image ls|grep javahello|grep cache|tr -s ' '

javahello cache ce8587c00465 1 second ago 262MB


In [17]:
docker run --rm javahello:cache

21:23:35.086 [main] INFO fr.univtln.bruno.demos.docker.App - Hello world maven + docker.


### Maven wrapper

Il est aussi possible de s'appuyer simplement sur l'image d'un JDK et d'utiliser un [wrapper maven](https://maven.apache.org/wrapper/). Dans ce cas, la bonne version de maven sera téléchargée au moment de la compilation.

```{literalinclude} sample-java/helloworld/Dockerfile.04.wrapper
:language: dockerfile
:caption: sample-java/helloworld/Dockerfile.04.wrapper
:name: sample-java/helloworld/Dockerfile.04.wrapper
```

In [20]:
(cd sample-java/helloworld && \
docker image build --quiet --tag javahello:wrapper --file Dockerfile.04.wrapper . )

sha256:4febce8b3a54b3154ea0829ce8aefd875b38db587153912f12cdf0d7417fd593


In [21]:
docker image ls|grep javahello|grep wrapper|tr -s ' '

javahello wrapper 4febce8b3a54 1 second ago 262MB


In [23]:
docker run --rm javahello:wrapper

21:24:35.239 [main] INFO fr.univtln.bruno.demos.docker.App - Hello world maven + docker.


### From scratch
Il est aussi possible réaliser une chaine de compilation personnalisée à partir de zéro par exemple avec [sdkman](https://sdkman.io/). 

```{literalinclude} sample-java/helloworld/Dockerfile.05.scratch
:language: dockerfile
:caption: sample-java/helloworld/Dockerfile.05.scratch
:name: sample-java/helloworld/Dockerfile.05.scratch
```

In [26]:
(cd sample-java/helloworld && \
docker image build --quiet --tag javahello:scratch --file Dockerfile.05.scratch . )

sha256:bd4891efcdd4569be845e5d084893ca4f782e14f2b765156c3c1c236b04de20d


In [27]:
docker image ls|grep javahello|grep scratch|tr -s ' '

javahello scratch bd4891efcdd4 Less than a second ago 262MB


In [28]:
docker run --rm javahello:scratch

21:25:51.771 [main] INFO fr.univtln.bruno.demos.docker.App - Hello world maven + docker.


### Avec les modules et jlink 

La meilleure solution est d'utiliser les modules de Java et jlink pour produire une livraison minimale du JRE et des dépendances pour le projet. 

**TODO**